In [37]:
import pandas as pd

pd.set_option('display.max_columns',None)

In [38]:
# text; location; description, followers_count; friends_count; listed_count; lang;

In [40]:
df=pd.read_json("train.json")


In [41]:

# get user information
df["user_id"]=df["user"].apply(lambda x: x["id"])
df["user_name"]=df["user"].apply(lambda x: x["name"])
df["user_location"]=df["user"].apply(lambda x: x["location"])
df["user_description"]=df["user"].apply(lambda x: x["description"])
df["user_created_at"]=df["user"].apply(lambda x: x["created_at"])
df["user_created_at"]=pd.to_datetime(df["user_created_at"])

# get the count
df["user_followers_count"]=df["user"].apply(lambda x: x["followers_count"])
df["user_friends_count"]=df["user"].apply(lambda x: x["friends_count"])
df["user_listed_count"]=df["user"].apply(lambda x: x["listed_count"])
df["user_favourites_count"]=df["user"].apply(lambda x: x["favourites_count"])
df["user_statuses_count"]=df["user"].apply(lambda x: x["statuses_count"])
df["user_lang"]=df["user"].apply(lambda x: x["lang"])

# get the hashtags and user mentions
df["hashtags"]=df["entities"].apply(lambda x: x["hashtags"])
df["hashtags_clean"]=df["hashtags"].apply(lambda x: [y["text"] for y in x] if len(x)!=0 else None)
df["user_mentions"]=df["entities"].apply(lambda x:x["user_mentions"])
df["user_mentions_user_id"]=df["user_mentions"].apply(lambda x: [y["id"] for y in x] if len(x)!=0 else None)
df["user_mentions_user_name"]=df["user_mentions"].apply(lambda x: [y["name"] for y in x] if len(x)!=0 else None)

# get the quoted information
df["quoted_status_created_at"]=df["quoted_status"].apply(lambda x: x["created_at"] if pd.isnull(x)==False else None)
df["quoted_status_created_at"]=pd.to_datetime(df["quoted_status_created_at"])
df["quoted_status_id"]=df["quoted_status"].apply(lambda x: x["id"] if pd.isnull(x)==False else None)
df["quoted_status_text"]=df["quoted_status"].apply(lambda x: x["text"] if pd.isnull(x)==False else None)
df["quoted_status_hashtags"]=df["quoted_status"].apply(lambda x: x["entities"]["hashtags"] if pd.isnull(x)==False else None)
df["quoted_status_hashtags_clean"]=df["quoted_status_hashtags"].apply(lambda x: [y["text"] for y in x] if (x is not None and len(x)!=0) else None)
df["quoted_status_user_mentions"]=df["quoted_status"].apply(lambda x: x["entities"]["user_mentions"] if pd.isnull(x)==False else None)
df["quoted_status_user_mentions_user_id"]=df["quoted_status_user_mentions"].apply(lambda x:[y["id"] for y in x] if (x is not None and len(x)!=0) else None)
df["quoted_status_user_mentions_user_name"]=df["quoted_status_user_mentions"].apply(lambda x:[y["name"] for y in x] if(x is not None and len(x)!=0) else None)

# get the retweet information
df["retweeted_status_created_at"]=df["retweeted_status"].apply(lambda x: x["created_at"] if pd.isnull(x)==False else None)
df["retweeted_status_created_at"]=pd.to_datetime(df["retweeted_status_created_at"])
df["retweeted_status_id"]=df["retweeted_status"].apply(lambda x: x["id"] if pd.isnull(x)==False else None)
df["retweeted_status_text"]=df["retweeted_status"].apply(lambda x: x["text"] if pd.isnull(x)==False else None)
df["retweeted_status_hashtags"]=df["retweeted_status"].apply(lambda x: x["entities"]["hashtags"] if pd.isnull(x)==False else None)
df["retweeted_status_hashtags_clean"]=df["retweeted_status_hashtags"].apply(lambda x: [y["text"] for y in x] if(x is not None and len(x)!=0) else None)
df["retweeted_status_user_mentions"]=df["retweeted_status"].apply(lambda x: x["entities"]["user_mentions"] if pd.isnull(x)==False else None)
df["retweeted_status_user_mentions_user_id"]=df["retweeted_status_user_mentions"].apply(lambda x:[y["id"] for y in x] if (x is not None and len(x)!=0) else None)
df["retweeted_status_user_mentions_user_name"]=df["retweeted_status_user_mentions"].apply(lambda x:[y["name"] for y in x] if (x is not None and len(x)!=0) else None)


In [42]:
import json
import numpy as np

types = set() # all types

entities = df["entities"]
for entity in entities:
  if entity is not np.NaN and "media" in entity:
    medias = entity["media"]
    for media in medias:
      if "type" in media:
        types.add(media["type"])

extended_entities = df["extended_entities"]
for entity in extended_entities:
  if entity is not np.NaN and "media" in entity:
    medias = entity["media"]
    for media in medias:
      if "type" in media:
        types.add(media["type"])

print(types)

{'photo', 'animated_gif', 'video'}


In [43]:
types = list(types)
types_idx = {t: i for i, t in enumerate(types)}
media_type_df = pd.DataFrame([[0 for _ in range(len(types))] for _ in range(df.shape[0])], columns=types)


rows_affected = []

entities = df["entities"]
for i,  entity in enumerate(entities):
  if entity is not np.NaN and "media" in entity:
    medias = entity["media"]
    for media in medias:
      if "type" in media:
        t = media["type"]
        media_type_df[t].iloc[i] = 1
        rows_affected += [i]

extended_entities = df["extended_entities"]
for i,  entity in enumerate(entities):
  if entity is not np.NaN and "media" in entity:
    medias = entity["media"]
    for media in medias:
      if "type" in media:
        t = media["type"]
        media_type_df[t].iloc[i] = 1
        rows_affected += [i]

In [44]:
rows_affected[:10]

[14, 95, 147, 229, 286, 302, 324, 482, 546, 582]

In [45]:
media_type_df.iloc[14,:]

photo           1
animated_gif    0
video           0
Name: 14, dtype: int64

In [46]:
df.head(1)

,created_at,id,id_str,text,truncated,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,possibly_sensitive,extended_entities,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status,user_id,user_name,user_location,user_description,user_created_at,user_followers_count,user_friends_count,user_listed_count,user_favourites_count,user_statuses_count,user_lang,hashtags,hashtags_clean,user_mentions,user_mentions_user_id,user_mentions_user_name,quoted_status_created_at,quoted_status_text,quoted_status_hashtags,quoted_status_hashtags_clean,quoted_status_user_mentions,quoted_status_user_mentions_user_id,quoted_status_user_mentions_user_name,retweeted_status_created_at,retweeted_status_id,retweeted_status_text,retweeted_status_hashtags,retweeted_status_hashtags_clean,retweeted_status_user_mentions,retweeted_status_user_mentions_user_id,retweeted_status_user_mentions_user_name
0,2018-07-09 10:26:27+00:00,1016267329137729536,1016267329137729536,"@Fhumu_Makhwanya Hi, no you cannot withdraw fu...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://www.lithium.com"" rel=""nofollo...",1.016267e+18,1.016267e+18,72326645.0,72326645.0,Fhumu_Makhwanya,"{'id': 56980888, 'id_str': '56980888', 'name':...",None,None,None,NaN,False,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN,56980888,Standard Bank SA,South Africa,Every action sets something in motion and that...,2009-07-15 10:21:06+00:00,120597,13723,409,9593,153810,en,[],None,"[{'screen_name': 'Fhumu_Makhwanya', 'name': 'F...",[72326645],[Fhumulani Makhwanya],NaT,None,None,None,None,None,None,NaT,NaN,None,None,None,None,None,None


In [47]:
# get the "quoted" tweets

df[~(df["quoted_status"].isnull())].head(2)

,created_at,id,id_str,text,truncated,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,possibly_sensitive,extended_entities,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status,user_id,user_name,user_location,user_description,user_created_at,user_followers_count,user_friends_count,user_listed_count,user_favourites_count,user_statuses_count,user_lang,hashtags,hashtags_clean,user_mentions,user_mentions_user_id,user_mentions_user_name,quoted_status_created_at,quoted_status_text,quoted_status_hashtags,quoted_status_hashtags_clean,quoted_status_user_mentions,quoted_status_user_mentions_user_id,quoted_status_user_mentions_user_name,retweeted_status_created_at,retweeted_status_id,retweeted_status_text,retweeted_status_hashtags,retweeted_status_hashtags_clean,retweeted_status_user_mentions,retweeted_status_user_mentions_user_id,retweeted_status_user_mentions_user_name
110,2018-07-11 10:01:42+00:00,1016985878474973184,1016985878474973184,RT https://t.co/fA2wOuAQhn,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://www.lithium.com"" rel=""nofollo...",NaN,NaN,NaN,NaN,None,"{'id': 56980888, 'id_str': '56980888', 'name':...",None,None,None,NaN,True,3,9,False,False,und,0.0,NaN,NaN,1.016942e+18,1.016942e+18,{'created_at': 'Wed Jul 11 07:07:34 +0000 2018...,56980888,Standard Bank SA,South Africa,Every action sets something in motion and that...,2009-07-15 10:21:06+00:00,120597,13723,409,9593,153810,en,[],None,[],None,None,2018-07-11 07:07:34+00:00,#MyFearlessNext Mentor @DeneBothaSA believes f...,"[{'text': 'MyFearlessNext', 'indices': [0, 15]}]",[MyFearlessNext],"[{'screen_name': 'DeneBothaSA', 'name': 'Dene ...",[488366553],[Dene Botha],NaT,NaN,None,None,None,None,None,None
191,2018-07-12 10:25:08+00:00,1017354163271938048,1017354163271938048,One of our #MyFearlessNext mentors @BulelaniBa...,True,"{'hashtags': [{'text': 'MyFearlessNext', 'indi...","<a href=""https://www.lithium.com"" rel=""nofollo...",NaN,NaN,NaN,NaN,None,"{'id': 56980888, 'id_str': '56980888', 'name':...",None,None,None,NaN,True,2,4,False,False,en,0.0,NaN,NaN,1.017348e+18,1.017348e+18,{'created_at': 'Thu Jul 12 10:02:10 +0000 2018...,56980888,Standard Bank SA,South Africa,Every action sets something in motion and that...,2009-07-15 10:21:06+00:00,120597,13723,409,9593,153810,en,"[{'text': 'MyFearlessNext', 'indices': [11, 26]}]",[MyFearlessNext],"[{'screen_name': 'BulelaniBala', 'name': '#Get...",[186573691],[#GetThingsDone],2018-07-12 10:02:10+00:00,"@StandardBankZA yes , there is . I have a few ...",[],None,"[{'screen_name': 'StandardBankZA', 'name': 'St...",[56980888],[Standard Bank SA],NaT,NaN,None,None,None,None,None,None


In [48]:
# get the "retweeted" tweets

df[~(df["retweeted_status"].isnull())].head(2)

,created_at,id,id_str,text,truncated,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,possibly_sensitive,extended_entities,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status,user_id,user_name,user_location,user_description,user_created_at,user_followers_count,user_friends_count,user_listed_count,user_favourites_count,user_statuses_count,user_lang,hashtags,hashtags_clean,user_mentions,user_mentions_user_id,user_mentions_user_name,quoted_status_created_at,quoted_status_text,quoted_status_hashtags,quoted_status_hashtags_clean,quoted_status_user_mentions,quoted_status_user_mentions_user_id,quoted_status_user_mentions_user_name,retweeted_status_created_at,retweeted_status_id,retweeted_status_text,retweeted_status_hashtags,retweeted_status_hashtags_clean,retweeted_status_user_mentions,retweeted_status_user_mentions_user_id,retweeted_status_user_mentions_user_name
21,2018-07-09 16:17:52+00:00,1016355767333605380,1016355767333605376,RT @SB_BizConnect: Laurel and Rachel embraced ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",NaN,NaN,NaN,NaN,None,"{'id': 56980888, 'id_str': '56980888', 'name':...",None,None,None,NaN,False,4,0,False,False,en,NaN,NaN,{'created_at': 'Mon Jul 09 16:13:49 +0000 2018...,NaN,NaN,NaN,56980888,Standard Bank SA,South Africa,Every action sets something in motion and that...,2009-07-15 10:21:06+00:00,120597,13723,409,9593,153810,en,[],None,"[{'screen_name': 'SB_BizConnect', 'name': 'SB ...",[18074661],[SB BizBanking],NaT,None,None,None,None,None,None,2018-07-09 16:13:49+00:00,1.016355e+18,Laurel and Rachel embraced their drive to own ...,[],None,[],None,None
142,2018-07-11 15:17:05+00:00,1017065246475857920,1017065246475857920,RT @IAFbrands: Media Alert:\n\nWe would like t...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 56980888, 'id_str': '56980888', 'name':...",None,None,None,NaN,True,3,0,False,False,en,NaN,NaN,{'created_at': 'Wed Jul 11 14:37:29 +0000 2018...,NaN,1.017029e+18,NaN,56980888,Standard Bank SA,South Africa,Every action sets something in motion and that...,2009-07-15 10:21:06+00:00,120597,13723,409,9593,153810,en,[],None,"[{'screen_name': 'IAFbrands', 'name': 'BEAT TH...","[2182460808, 186573691, 56980888]","[BEAT THE BRAND-U-WORRY, #GetThingsDone, Stand...",NaT,None,None,None,None,None,None,2018-07-11 14:37:29+00:00,1.017055e+18,Media Alert:\n\nWe would like to congratulate ...,[],None,"[{'screen_name': 'BulelaniBala', 'name': '#Get...","[186573691, 56980888]","[#GetThingsDone, Standard Bank SA]"


In [49]:
#clean result 1
clean_result1=df[[
    # tweet info
    'created_at',
    'id', 
    'text',
    'in_reply_to_status_id',
    'in_reply_to_user_id', 
    'in_reply_to_screen_name',
    'geo',
    'coordinates', 
    'place',
    'retweet_count', 
    'favorite_count',
    'favorited', 
    'retweeted', 
    'lang',

    # user info
    'user_id',
    'user_name',
    'user_location', 
    'user_description',
    'user_created_at',
    'user_followers_count',
    'user_friends_count', 
    'user_listed_count',
    'user_favourites_count',
    'user_statuses_count', 
    'user_lang', 
    'hashtags_clean',
    'user_mentions_user_id',
    'user_mentions_user_name',

    # quote info
    'is_quote_status', 
    'quoted_status',
    'quoted_status_id',
    'quoted_status_created_at',
    'quoted_status_text', 
    'quoted_status_hashtags_clean', 
    'quoted_status_user_mentions_user_id',
    'quoted_status_user_mentions_user_name',

    # retweet info
    'retweeted_status',
    'retweeted_status_created_at',
    'retweeted_status_id',
    'retweeted_status_text',
    'retweeted_status_hashtags_clean',
    'retweeted_status_user_mentions_user_id',
    'retweeted_status_user_mentions_user_name'
]]

clean_result1.to_csv("clean_result1.csv",index=False)

In [50]:
# clean result 3

clean_result3=df[[
    'user_friends_count', 
    'user_followers_count',
    'user_created_at',
    'favorite_count',
    "user_statuses_count"
]] 
clean_result3.to_csv("clean_result3.csv",index=False)


In [51]:
clean_result3=pd.read_csv("clean_result3.csv")
df1 = pd.DataFrame(clean_result3)
df1[:5]
#change "user_created_at" from string type to int64 type
from datetime import datetime
df1['user_created_at'] = pd.to_datetime(df.user_created_at).dt.tz_localize(None).astype(np.int64)

df1 = pd.concat([df1, media_type_df], axis=1)

df1.info()
df1.to_csv('final_data.csv', encoding='utf-8')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96562 entries, 0 to 96561
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   user_friends_count    96562 non-null  int64
 1   user_followers_count  96562 non-null  int64
 2   user_created_at       96562 non-null  int64
 3   favorite_count        96562 non-null  int64
 4   user_statuses_count   96562 non-null  int64
 5   photo                 96562 non-null  int64
 6   animated_gif          96562 non-null  int64
 7   video                 96562 non-null  int64
dtypes: int64(8)
memory usage: 5.9 MB


/var/folders/pp/1gzclzqx4ldd418wy5h99v8w0000gn/T/ipykernel_18121/2811078255.py:6: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  df1['user_created_at'] = pd.to_datetime(df.user_created_at).dt.tz_localize(None).astype(np.int64)


In [52]:
#turn into vector
import csv
results = []
with open("final_data.csv",'r', encoding='utf-8', errors='ignore') as csvfile:
    next(csvfile)
    reader = csv.reader(csvfile,quoting=csv.QUOTE_NONNUMERIC) # change contents to floats
    for row in reader:# each row is a list
        results.append(row)

In [53]:
print(results[:2])

[[0.0, 13723.0, 120597.0, 1.247653266e+18, 0.0, 153810.0, 0.0, 0.0, 0.0], [1.0, 13723.0, 120597.0, 1.247653266e+18, 0.0, 153810.0, 0.0, 0.0, 0.0]]
